<a href="https://colab.research.google.com/github/mjag7682/CS9-1-NLP-for-Twitter-Data-for-predicting-stocks/blob/master/PreprocessingFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install, Import, Mount statements

In [1]:
!pip install contractions
!pip install emoji
!pip install ekphrasis

import sys
import math
import pandas as pd
import re
import emoji
import nltk
import contractions
import torch

nltk.download('punkt')
nltk.download('stopwords')
# from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer 
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize 
from nltk.tokenize.treebank import TreebankWordDetokenizer
import requests 
from pprint import pprint
import numpy as np
from nltk.stem import PorterStemmer

import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras
# import bert
# from bert import run_classifier
# from bert import optimization
# from bert import tokenization
import os
import pprint
import json
import random
import string
import sys
from ekphrasis.classes.segmenter import Segmenter
import itertools


     |████████████████████████████████| 317kB 13.0MB/s 
     |████████████████████████████████| 245kB 42.0MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81698 sha256=5b9f31992a58143cd2196c4bf8b9c9146f9f059c397e2567cd79ab13e29dcbf2
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
     |████████████████████████████████| 51kB 4.7MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=2dcc4fd17dc129eebe5e44402a5f00fdfd0e75e52f7009c3765ed6597b6f3d24
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji
     |████████████████████████████████| 81kB 6.5MB/s 
     |████████████████████████████████| 184kB 18.5MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-cp36-none-any.whl

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load file

In [3]:
df_comb = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Capstone/Labelled (Binary - same day open close)/labelled_DIA.csv')
combine_ds = df_comb.sample(frac=1)
combine_ds

,Unnamed: 0,symbol,message,datetime,user,message_id,Date,Time,Day_of_week,polarity
16590,16590,DIA,$DIA BETS on market tomorrow?,2020-04-07 18:12:46,1117334,205129040,2020-04-07,18:12:46,Tuesday,0
154988,154988,DIA,Municipal bond funds reported an inflow for th...,2014-10-17 12:15:41,237666,28110605,2014-10-17,12:15:41,Friday,1
174141,174141,DIA,TS TradePlan for YM http://stks.co/hbRm - Frid...,2013-06-28 08:55:34,13803,14345522,2013-06-28,08:55:34,Friday,0
94680,94680,DIA,$SPX $SPY $QQQ $NDX $IWM $VIX $DIA $DJIA NASDA...,2018-03-14 17:18:12,917666,116238736,2018-03-14,17:18:12,Wednesday,0
64506,64506,DIA,"Although the technical rating is bad, $DIA doe...",2018-12-19 15:39:54,47688,148382242,2018-12-19,15:39:54,Wednesday,0
...,...,...,...,...,...,...,...,...,...,...
166025,166025,DIA,$DIA overly bearish sentiment tells me we coul...,2014-02-03 20:08:06,170632,19676228,2014-02-03,20:08:06,Monday,0
182008,182008,DIA,If $SPY $DIA $QQQ continue to fall. Look for b...,2012-09-26 15:08:40,161307,9693821,2012-09-26,15:08:40,Wednesday,0
36692,36692,DIA,$SPY $DIA green \n \nSo looks like the rever...,2019-11-21 19:36:44,349810,184600235,2019-11-21,19:36:44,Thursday,0
144330,144330,DIA,&quot;@ep_capital: $INTC -0.70% the lone decli...,2015-07-09 14:02:15,8115,39428873,2015-07-09,14:02:15,Thursday,0


## Preprocess data

In [8]:
# fill nan values in file with '0'
combine_ds.isna().values.any()
combine_ds['message'] = combine_ds['message'].fillna('0')

In [9]:
combine_ds['message'] = combine_ds['message'].str.lower()
message = combine_ds['message'].tolist()

print(message[:10])

['$dia bets on market tomorrow?', 'municipal bond funds reported an inflow for the 23rd time in the past 24wks (+$444mn). $dia $qqq $spy $mub $hyd $tlt $tbt', 'ts tradeplan for ym http://stks.co/hbrm - friday june 28 2013 $es_f $ym_f $6e_f $cl_f $gc_f sent to members $dia $indu #futures #trading', '$spx $spy $qqq $ndx $iwm $vix $dia $djia nasdaq market internals summary', 'although the technical rating is bad, $dia does present a nice setup opportunity. https://www.chartmill.com/analyze.php?utm_source=stocktwits&amp;utm_medium=ta&amp;utm_content=setup&amp;utm_campaign=social_tracking#/dia?key=810cb939-dc70-485b-b7ec-a9a1fe911f70', 'new hod on $tick on that move over 15459 as well $dia $indu', '$iwm $spy $qqq $dia $spx markets makers move prices flat into the sun. vol. event. as close to middle as possible.', '$dia  they will not rollback tariffs . c&#39;mon look at  the news man! the asian markets usa everyone this will be no deal!!!!', '$spy $dia panic selling in overall markets howev

In [10]:
def remove_stopwords(msg):
    filtered_sentence = [w for w in msg_tokens if not w in stop_words]
    return filtered_sentence

def remove_punctuation_re(x):
    x = ' '.join(re.sub("https?://\S+","",x).split())     #Removing URLs

    x = ' '.join(re.sub("^@\S+|\s@\S+","",x).split())     #Removing Mentions

    # x = ' '.join(re.sub(r'[^$\w\s]'," ",x).split())
    x = ' '.join(re.sub(r'[^\w\s]'," ",x).split())        #Removes Hashtags

    x = ' '.join(re.sub(r'_'," ",x).split())              #Removing _ from emojis text

    return x

# replace repeating letter
def rpt_replace(match):
    # print(match.group(1))
    return match.group(1)+match.group(1)

# substitute original word with replaced word, if any
def processRepeatings(data):
    re_t= re.sub(message_rpt, rpt_replace, data )
    # print(re_t)
    return re_t

In [11]:
print(sys.getrecursionlimit())
sys.setrecursionlimit(1800)
print(sys.getrecursionlimit())

1000
1800


In [12]:
stop_words = sw.words("english")
tweet_tokenizer = TweetTokenizer()
detokenizer = TreebankWordDetokenizer()
# message_p = []

# for repeating characters in words
message_rpt = re.compile(r"(.)\1{2,}", re.IGNORECASE)

# segmenter using the word statistics from Twitter
seg_tw = Segmenter(corpus="twitter")


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt


In [13]:
message_p = []
for msg in message:

    if msg == '0': #nan replaced by '0'
      message_p.append('-1')

    else:
      # remove emojis
      msg = emoji.demojize(msg)

      # fix contractions
      msg = contractions.fix(msg)

      # remove punctuations
      msg = remove_punctuation_re(msg) 

      #tokenize
      msg_tokens = tweet_tokenizer.tokenize(msg)

      #For Hashtags elongated words using Word segmenter
      message_seg = []
      for w in msg_tokens:
        message_seg.append(seg_tw.segment(w))

      # remove stopwords
      msg = remove_stopwords(message_seg)

      if 'rt' in msg:
        # remove retweets
        message_p.append('-1')
      else: 
        # detokenize
        msg = detokenizer.detokenize(msg)

        # removing repeating characters like hurrrryyyyyy-- worrks on tokenized list
        msg = processRepeatings(msg)

        message_p.append(msg)


In [14]:
message_p[:10]

['dia bets market tomorrow',
 'municipal bond funds reported inflow 23rd time past 24wks 44mn dia qq spy mub hyd tlt tbt',
 'ts tradeplan ym friday june 28 2013 es f ym f 6e f cl f gc f sent members dia indu futures trading',
 'spx spy qq ndx iwm vix dia djia nasdaq market internals summary',
 'although technical rating bad dia present nice setup opportunity',
 'new hod tick move 15459 well dia indu',
 'iwm spy qq dia spx markets makers move prices flat sun vol event close middle possible',
 'dia rollback tariffs c 39 mon look news man asian markets usa everyone deal',
 'spy dia panic selling overall markets however stocks selling continue focus stronger positions livetradepro',
 'bearish persistence aaii survey via spy dia']

In [15]:
combine_ds['message'] = message_p
combine_ds

,Unnamed: 0,symbol,message,datetime,user,message_id,Date,Time,Day_of_week,polarity
16590,16590,DIA,dia bets market tomorrow,2020-04-07 18:12:46,1117334,205129040,2020-04-07,18:12:46,Tuesday,0
154988,154988,DIA,municipal bond funds reported inflow 23rd time...,2014-10-17 12:15:41,237666,28110605,2014-10-17,12:15:41,Friday,1
174141,174141,DIA,ts tradeplan ym friday june 28 2013 es f ym f ...,2013-06-28 08:55:34,13803,14345522,2013-06-28,08:55:34,Friday,0
94680,94680,DIA,spx spy qq ndx iwm vix dia djia nasdaq market ...,2018-03-14 17:18:12,917666,116238736,2018-03-14,17:18:12,Wednesday,0
64506,64506,DIA,although technical rating bad dia present nice...,2018-12-19 15:39:54,47688,148382242,2018-12-19,15:39:54,Wednesday,0
...,...,...,...,...,...,...,...,...,...,...
166025,166025,DIA,dia overly bearish sentiment tells could see b...,2014-02-03 20:08:06,170632,19676228,2014-02-03,20:08:06,Monday,0
182008,182008,DIA,spy dia qq continue fall look biotechs like am...,2012-09-26 15:08:40,161307,9693821,2012-09-26,15:08:40,Wednesday,0
36692,36692,DIA,spy dia green looks like reversal day release ...,2019-11-21 19:36:44,349810,184600235,2019-11-21,19:36:44,Thursday,0
144330,144330,DIA,quot ep capital intc 0 70 lone decliner dji di...,2015-07-09 14:02:15,8115,39428873,2015-07-09,14:02:15,Thursday,0


In [16]:
combine_ds['polarity'].value_counts()

1    99652
0    90033
Name: polarity, dtype: int64

In [17]:
# Drop Retweets
combine_ds.drop(combine_ds[combine_ds['message'] == '-1'].index, inplace = True) 
combine_ds

,Unnamed: 0,symbol,message,datetime,user,message_id,Date,Time,Day_of_week,polarity
16590,16590,DIA,dia bets market tomorrow,2020-04-07 18:12:46,1117334,205129040,2020-04-07,18:12:46,Tuesday,0
154988,154988,DIA,municipal bond funds reported inflow 23rd time...,2014-10-17 12:15:41,237666,28110605,2014-10-17,12:15:41,Friday,1
174141,174141,DIA,ts tradeplan ym friday june 28 2013 es f ym f ...,2013-06-28 08:55:34,13803,14345522,2013-06-28,08:55:34,Friday,0
94680,94680,DIA,spx spy qq ndx iwm vix dia djia nasdaq market ...,2018-03-14 17:18:12,917666,116238736,2018-03-14,17:18:12,Wednesday,0
64506,64506,DIA,although technical rating bad dia present nice...,2018-12-19 15:39:54,47688,148382242,2018-12-19,15:39:54,Wednesday,0
...,...,...,...,...,...,...,...,...,...,...
166025,166025,DIA,dia overly bearish sentiment tells could see b...,2014-02-03 20:08:06,170632,19676228,2014-02-03,20:08:06,Monday,0
182008,182008,DIA,spy dia qq continue fall look biotechs like am...,2012-09-26 15:08:40,161307,9693821,2012-09-26,15:08:40,Wednesday,0
36692,36692,DIA,spy dia green looks like reversal day release ...,2019-11-21 19:36:44,349810,184600235,2019-11-21,19:36:44,Thursday,0
144330,144330,DIA,quot ep capital intc 0 70 lone decliner dji di...,2015-07-09 14:02:15,8115,39428873,2015-07-09,14:02:15,Thursday,0


In [18]:
combine_ds['polarity'].value_counts()

1    99069
0    89530
Name: polarity, dtype: int64

In [ ]:
# Uncomment for FB
# FB_IPO = '2012-05-21'
# combine_ds.drop(combine_ds[combine_ds['Date'] < '2012-05-21'].index, inplace = True) 
# combine_ds

In [ ]:
# Uncomment for TSLA
# TSLA_IPO = '2010-06-30'
# combine_ds.drop(combine_ds[combine_ds['Date'] < TSLA_IPO].index, inplace = True) 
# combine_ds

In [ ]:
# combine_ds['polarity'].value_counts()

In [19]:
combine_ds.to_csv('prep_WMT1.csv')
!cp prep_WMT1.csv "/content/drive/My Drive"